## Long-Short Price & Volume Strategy

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
plt.style.use("seaborn-v0_8")


### Get Data

In [49]:
url = "https://91j3dag4m9.execute-api.us-east-1.amazonaws.com/master/getCMCPriceHistory/1?startDate=2021-01-01"
response = requests.get(url)
data = response.json()
table = []
# Response gives us o,h,l,c,v,mcap keys which we need to change our column names like open, close etc...
# because thats what our utility functions accept such as our technical indicator functions
for entry in data["priceHistory"]:
    table.append([
        entry["d"],
        entry["c"],
        entry["v"],
    ])

# Create a DataFrame from the extracted data
columns = ["date", "close", "volume"]
prices_df = pd.DataFrame(table, columns=columns)
prices_df.set_index("date", inplace=True)
prices_df['log_returns'] = np.log(prices_df['close'] / prices_df['close'].shift(1))
prices_df['volume_change'] = np.log(prices_df['volume'] / prices_df['volume'].shift(1))

# Remove outliers
prices_df.loc[prices_df['volume_change'] > 3, "volume_change"] = np.nan
prices_df.loc[prices_df['volume_change'] < -3, "volume_change"] = np.nan

prices_df["position"] = 0 # Trading position -> Neutral for all bars



Buy and go long (position = 1) if most recent returns har highly negative (conditionOne) and trading volume decreased (conditionTwo)

In [50]:
# Get returns threshold for highly negative returns (<= 10th percentile)
low_return_threshold = np.percentile(prices_df['log_returns'].dropna(), 10)
low_return_threshold

prices_df['condition_one'] = prices_df['log_returns'] <= low_return_threshold
prices_df

,close,volume,log_returns,volume_change,position,condition_one
date,,,,,,
2021-01-01,29374.151889,4.073030e+10,NaN,NaN,0,False
2021-01-02,32127.267939,6.786542e+10,0.089590,0.510554,0,False
2021-01-03,32782.024466,7.866524e+10,0.020175,0.147675,0,False
2021-01-04,31971.913522,8.116348e+10,-0.025023,0.031264,0,False
2021-01-05,33992.429344,6.754732e+10,0.061280,-0.183637,0,False
...,...,...,...,...,...,...
2023-09-05,25779.981795,1.109474e+10,-0.001257,0.038039,0,False
2023-09-06,25753.236258,1.275271e+10,-0.001038,0.139272,0,False
2023-09-07,26240.194892,1.108831e+10,0.018732,-0.139852,0,False


In [51]:
# Getting volume change threshold for (moderate) volume decreases (between 5th and 20th percentile)
volume_threshold = np.percentile(prices_df['volume_change'].dropna(), [2, 20])
volume_threshold

prices_df['condition_two'] = prices_df['volume_change'].between(volume_threshold[0], volume_threshold[1])
prices_df

,close,volume,log_returns,volume_change,position,condition_one,condition_two
date,,,,,,,
2021-01-01,29374.151889,4.073030e+10,NaN,NaN,0,False,False
2021-01-02,32127.267939,6.786542e+10,0.089590,0.510554,0,False,False
2021-01-03,32782.024466,7.866524e+10,0.020175,0.147675,0,False,False
2021-01-04,31971.913522,8.116348e+10,-0.025023,0.031264,0,False,False
2021-01-05,33992.429344,6.754732e+10,0.061280,-0.183637,0,False,False
...,...,...,...,...,...,...,...
2023-09-05,25779.981795,1.109474e+10,-0.001257,0.038039,0,False,False
2023-09-06,25753.236258,1.275271e+10,-0.001038,0.139272,0,False,False
2023-09-07,26240.194892,1.108831e+10,0.018732,-0.139852,0,False,False


In [52]:
condition_one = prices_df['condition_one']
condition_two = prices_df['condition_two']

prices_df.loc[condition_one & condition_two, "position"] = 1
prices_df

,close,volume,log_returns,volume_change,position,condition_one,condition_two
date,,,,,,,
2021-01-01,29374.151889,4.073030e+10,NaN,NaN,0,False,False
2021-01-02,32127.267939,6.786542e+10,0.089590,0.510554,0,False,False
2021-01-03,32782.024466,7.866524e+10,0.020175,0.147675,0,False,False
2021-01-04,31971.913522,8.116348e+10,-0.025023,0.031264,0,False,False
2021-01-05,33992.429344,6.754732e+10,0.061280,-0.183637,0,False,False
...,...,...,...,...,...,...,...
2023-09-05,25779.981795,1.109474e+10,-0.001257,0.038039,0,False,False
2023-09-06,25753.236258,1.275271e+10,-0.001038,0.139272,0,False,False
2023-09-07,26240.194892,1.108831e+10,0.018732,-0.139852,0,False,False


Sell and go short (position = -1) if most revent returns are highly positive (conditionThree) and trading volume decreased

In [54]:
high_return_threshold = np.percentile(prices_df['log_returns'].dropna(), 90)
high_return_threshold

prices_df['condition_three'] = prices_df['log_returns'] >= high_return_threshold
prices_df.loc[prices_df['condition_three'] & prices_df['condition_two'], "position"] = -1


prices_df.position.value_counts()

position
 0    974
-1      5
 1      3
Name: count, dtype: int64